<a href="https://colab.research.google.com/github/meerilahi/algoTrading/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
%matplotlib inline

In [ ]:
API_KEY = "XM0ZE0M05HEDFVTP"
ts = TimeSeries(key = API_KEY, output_format = 'pandas')
data = ts.get_daily(symbol = 'MSFT', outputsize = 'full')[0]

In [268]:
df = data[['4. close', '5. volume']]
df = df.rename(columns = {'4. close':'target', '5. volume': 'volume'})
df = df[::-1]
for i in range(1,31):
    df[f'close_lag_{i}'] = df['target'].shift(i)
    df[f'volume_lag_{i}'] = df['volume'].shift(i)
df = df.dropna()
df = df.iloc[::,::-1]
scaler = MinMaxScaler()
df.iloc[:,0:-2] = scaler.fit_transform(df.iloc[:,0:-2])
df

,volume_lag_30,close_lag_30,volume_lag_29,close_lag_29,volume_lag_28,close_lag_28,volume_lag_27,close_lag_27,volume_lag_26,close_lag_26,volume_lag_25,close_lag_25,volume_lag_24,close_lag_24,volume_lag_23,close_lag_23,volume_lag_22,close_lag_22,volume_lag_21,close_lag_21,volume_lag_20,close_lag_20,volume_lag_19,close_lag_19,volume_lag_18,close_lag_18,volume_lag_17,close_lag_17,volume_lag_16,close_lag_16,volume_lag_15,close_lag_15,volume_lag_14,close_lag_14,volume_lag_13,close_lag_13,volume_lag_12,close_lag_12,volume_lag_11,close_lag_11,volume_lag_10,close_lag_10,volume_lag_9,close_lag_9,volume_lag_8,close_lag_8,volume_lag_7,close_lag_7,volume_lag_6,close_lag_6,volume_lag_5,close_lag_5,volume_lag_4,close_lag_4,volume_lag_3,close_lag_3,volume_lag_2,close_lag_2,volume_lag_1,close_lag_1,volume,target
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-12-14,0.035509,0.170686,0.029603,0.171106,0.028038,0.169868,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,0.032222,0.163657,0.030228,0.158816,0.040553,0.159501,0.047093,0.154395,0.045105,0.154263,0.039751,0.156606,0.067408,0.165027,0.050483,0.164607,0.035944,0.169183,0.014365,0.167923,0.033970,0.165867,0.044809,0.167768,0.031752,0.172498,0.037399,0.176079,0.044770,0.178975,0.031952,0.177472,0.038583,0.172078,0.024995,0.169315,0.029525,0.171526,0.021475,0.174001,0.035022,0.180080,72058100.0,98.69
1999-12-15,0.029603,0.171106,0.028038,0.169868,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,0.032222,0.163657,0.030228,0.158816,0.040553,0.159501,0.047093,0.154395,0.045105,0.154263,0.039751,0.156606,0.067408,0.165027,0.050483,0.164607,0.035944,0.169183,0.014365,0.167923,0.033970,0.165867,0.044809,0.167768,0.031752,0.172498,0.037399,0.176079,0.044770,0.178975,0.031952,0.177472,0.038583,0.172078,0.024995,0.169315,0.029525,0.171526,0.021475,0.174001,0.035022,0.180080,0.113136,0.184656,77785800.0,108.44
1999-12-16,0.028038,0.169868,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,0.032222,0.163657,0.030228,0.158816,0.040553,0.159501,0.047093,0.154395,0.045105,0.154263,0.039751,0.156606,0.067408,0.165027,0.050483,0.164607,0.035944,0.169183,0.014365,0.167923,0.033970,0.165867,0.044809,0.167768,0.031752,0.172498,0.037399,0.176079,0.044770,0.178975,0.031952,0.177472,0.038583,0.172078,0.024995,0.169315,0.029525,0.171526,0.021475,0.174001,0.035022,0.180080,0.113136,0.184656,0.122923,0.206207,64330100.0,113.69
1999-12-17,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,0.032222,0.163657,0.030228,0.158816,0.040553,0.159501,0.047093,0.154395,0.045105,0.154263,0.039751,0.156606,0.067408,0.165027,0.050483,0.164607,0.035944,0.169183,0.014365,0.167923,0.033970,0.165867,0.044809,0.167768,0.031752,0.172498,0.037399,0.176079,0.044770,0.178975,0.031952,0.177472,0.038583,0.172078,0.024995,0.169315,0.029525,0.171526,0.021475,0.174001,0.035022,0.180080,0.113136,0.184656,0.122923,0.206207,0.099930,0.217811,52949400.0,115.25
1999-12-20,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,0.032222,0.163657,0.030228,0.158816,0.040553,0.159501,0.047093,0.154395,0.045105,0.154263,0.039751,0.156606,0.067408,0.165027,0.050483,0.164607,0.035944,0.169183,0.014365,0.167923,0.033970,0.165867,0.044809,0.167768,0.031752,0.172498,0.037399,0.176079,0.044770,0.178975,0.031952,0.177472,0.038583,0.172078,0.024995,0.169315,0.029525,0.171526,0.021475,0.174001,0.035022,0.180080,0.113136,0.184656,0.122923,0.206207,0.099930,0.217811,0.080483,0.221259,19716900.0,112.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-10,0.016573,0.948056,0

In [274]:
X = df.iloc[:,:-2].to_numpy()
X = X.reshape(6351,30,2)
y = df.iloc[:,-1].to_numpy()
X.shape, y.shape

((6351, 30, 2), (6351,))

In [281]:
split_index = int(len(X) * 0.8)
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [284]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape = (30, 2)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse', metrics=['mean_absolute_error'])
model.fit(train_ds.batch(32), epochs = 20,validation_data = test_ds.batch(32))

Epoch 1/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - loss: 2116.0818 - mean_absolute_error: 31.1676 - val_loss: 28189.5391 - val_mean_absolute_error: 147.7904
Epoch 2/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - loss: 1204.7015 - mean_absolute_error: 22.8839 - val_loss: 43687.5469 - val_mean_absolute_error: 193.0732
Epoch 3/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 1004.7732 - mean_absolute_error: 24.3288 - val_loss: 51862.3945 - val_mean_absolute_error: 213.1945
Epoch 4/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - loss: 843.3767 - mean_absolute_error: 23.4488 - val_loss: 54590.6523 - val_mean_absolute_error: 219.4997
Epoch 5/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 782.9937 - mean_absolute_error: 22.9750 - val_loss: 55307.2969 - val_mean_absolute_error: 221.1260
Epoch 6/20
159/159 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 784.0012 - mean_absolute_error: 23.3980 - val_loss: 58667.8477 - val_mean_absolute_error: 228.5984
Epoch 7/20
159/159 ━━━━━━━━━━━━━━━━━━

In [285]:
model.save("model_copy.keras")
model2 = tf.keras.models.load_model("model_copy.keras")

In [287]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(model2.predict(X_test[i].reshape(1,30,2)))
    print('Example' + str(i))
    print('y_pred: ' + str(y_pred[i]) + ' -  y_test: ' + str(y_test[i]))
    model2.fit(X_test[i].reshape(1,30,2), y_test[i].reshape(1,1), epochs = 1)

Streaming output truncated to the last 5000 lines.
Example21
y_pred: [[129.16144]] -  y_test: 146.92
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 315.3665 - mean_absolute_error: 17.7586
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Example22
y_pred: [[132.38867]] -  y_test: 156.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 562.7014 - mean_absolute_error: 23.7213
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Example23
y_pred: [[135.57678]] -  y_test: 149.7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 199.4652 - mean_absolute_error: 14.1232
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Example24
y_pred: [[138.63799]] -  y_test: 160.23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 466.2149 - mean_absolute_error: 21.5920
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Example25
y_pred: [[141.6635]] -  y_test: 157.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 257.4904 - mean_absolute_error: 16.0465
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Example26
y_pred: [[144.60207]] -  y_test: 152.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97m

In [ ]:
y_pred = np.array(y_pred)
y_pred = y_pred.reshape(y_pred.shape[0])

In [220]:
backtest = pd.DataFrame({
  "today_close" : y_test,
  "tommorrow_pred" : y_pred
})
backtest['tommorrow_pred'] = backtest['tommorrow_pred'].shift(-1)
backtest['navie_pred'] = backtest['today_close'].shift(1)
backtest["signal"] = backtest["tommorrow_pred"] > backtest["today_close"]
backtest["navie_signal"] = backtest["navie_pred"] > backtest["today_close"]
backtest.dropna(inplace = True)
backtest.head()

,today_close,tommorrow_pred,navie_pred,signal,navie_signal
1,170.17,166.446518,168.07,False,False
2,158.18,163.585403,170.17,True,True
3,162.01,167.247986,158.18,True,False
4,172.79,170.043030,162.01,False,False
5,164.51,169.537415,172.79,True,True


In [237]:
# mean_absolute_error(y_test, y_pred)
mean_absolute_error(y_test[:-1], y_test[1:])

3.7768031496062986

In [222]:
initial_balance = 10000
balance = initial_balance
shares = 0

for i in range(len(backtest) - 1):
    price = backtest["today_close"].iloc[i]

    if price > 0:  # Ensure valid price
        if backtest["navie_signal"].iloc[i] and balance > 0:  # Buy only if we have cash
            shares = balance / price
            balance = 0
        elif not backtest["navie_signal"].iloc[i] and shares > 0:  # Sell only if we have shares
            balance = shares * price
            shares = 0

# Final balance calculation
final_price = backtest["today_close"].iloc[-1]
final_balance = balance + (shares * final_price)  # Sell remaining shares if any
profit = final_balance - initial_balance

print(f"Final Balance: ${final_balance:.2f}, Profit: ${profit:.2f}")



Final Balance: $37286.16, Profit: $27286.16
